# علم البيانات في السحابة: طريقة "Azure ML SDK"

## المقدمة

في هذا الدفتر، سنتعلم كيفية استخدام Azure ML SDK لتدريب نموذج، نشره، واستهلاكه من خلال Azure ML.

المتطلبات المسبقة:
1. قمت بإنشاء مساحة عمل Azure ML.
2. قمت بتحميل [مجموعة بيانات فشل القلب](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) إلى Azure ML.
3. قمت برفع هذا الدفتر إلى Azure ML Studio.

الخطوات التالية هي:

1. إنشاء تجربة في مساحة عمل موجودة.
2. إنشاء مجموعة حوسبة.
3. تحميل مجموعة البيانات.
4. إعداد AutoML باستخدام AutoMLConfig.
5. تشغيل تجربة AutoML.
6. استكشاف النتائج والحصول على أفضل نموذج.
7. تسجيل أفضل نموذج.
8. نشر أفضل نموذج.
9. استهلاك نقطة النهاية.

## استيرادات خاصة بـ Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## تهيئة مساحة العمل  
تهيئة كائن مساحة العمل من التكوين المحفوظ. تأكد من وجود ملف التكوين في .\config.json  


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## إنشاء تجربة Azure ML

لنقم بإنشاء تجربة باسم 'aml-experiment' في مساحة العمل التي قمنا بتفعيلها للتو.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## إنشاء مجموعة حوسبة  
ستحتاج إلى إنشاء [هدف الحوسبة](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) لتشغيل AutoML الخاص بك.  


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## البيانات  
تأكد من أنك قد قمت برفع مجموعة البيانات إلى Azure ML وأن المفتاح يحمل نفس اسم مجموعة البيانات.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## إعداد AutoML


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## تشغيل AutoML


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## نشر أفضل نموذج

قم بتشغيل الكود التالي لنشر أفضل نموذج. يمكنك رؤية حالة النشر في بوابة Azure ML. قد تستغرق هذه الخطوة بضع دقائق.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## استخدام نقطة النهاية
يمكنك إضافة المدخلات إلى نموذج الإدخال التالي.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالترجمة البشرية الاحترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
